<a href="https://colab.research.google.com/github/ShadmanRohan/n-gram-language-model/blob/master/ngam-language-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### Embedding Layer in PyTorch

In [2]:
word_to_ix = {"hello": 0, "world": 1} # put two words in a dictionary
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["world"]], dtype=torch.long) # torch.long is an Integer, returns the index
print(lookup_tensor)
hello_embed = embeds(lookup_tensor) # callable returns the vector in the index
print(hello_embed) # randomly generated vector

tensor([1])
tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


In [3]:
embedding = nn.Embedding(1000,128) # randomly initialised matrix, 10000 words * 128 dim each
embedding(torch.LongTensor([3,6])) # return 128 dim vector

tensor([[-2.8895e-01,  1.5219e-01, -2.9803e-01, -1.3135e-01,  1.5365e+00,
          9.1930e-01,  3.4535e-01, -5.5059e-01, -9.6037e-01,  2.7690e-01,
         -6.8011e-01, -5.4579e-01,  1.1528e+00, -3.9975e-01,  9.7380e-01,
          1.5453e+00, -3.7614e-01, -2.4107e+00, -1.2778e+00, -6.2887e-02,
         -1.2308e+00, -1.0655e+00, -3.8646e-01,  3.7823e-01, -2.0760e-01,
          6.9363e-01,  4.1949e-01,  2.2524e+00, -8.0162e-02, -1.3489e+00,
         -8.2630e-01,  2.6483e-01, -1.6050e+00, -1.0645e-01,  2.4657e-01,
          6.1251e-01, -7.6937e-03,  2.6158e+00, -6.3858e-02,  2.6087e-01,
         -3.6079e-01, -1.2617e+00,  1.9147e+00, -1.8613e+00,  4.7390e-01,
          9.9882e-01, -1.1198e-01, -1.7404e+00, -4.2655e-01,  9.8075e-01,
          1.8589e+00, -1.0920e+00, -7.1291e-01, -6.3866e-02,  1.0757e+00,
         -5.5361e-01, -3.2686e+00,  4.7499e-01, -2.1142e+00, -1.5002e+00,
          2.6929e-01, -4.8861e-01,  1.3694e+00,  4.5392e-01,  1.1753e+00,
         -3.4212e-01, -3.8716e-01,  5.

### Train a Tri-gram Language Model

#### 1. Prepare data

In [4]:
CONTEXT_SIZE = 2 # parameters
EMBEDDING_DIM = 10 # word embedding dimension
# Some random bangla news paper article
test_sentence = """প্রথমেই শুরু হয়েছে স্বাস্থ্য ও চিকিৎসার সংকট। এতে সরকারকে জরুরি স্বাস্থ্য বরাদ্দ বাড়াতে হবে। অন্যথায় স্বাস্থ্য ও চিকিৎসার 
সংকট প্রলম্বিত হয়ে অনুৎপাদনশীলতার জন্ম হবে, যার আর্থিক দায় ব্যাপক। দ্বিতীয় সংকট হতে পারে খাদ্য ও মানবিক সংকট। যেহেতু সংক্রমণ বিস্তার 
রোধে প্রায় পূর্ণাঙ্গ ‘লকডাউন’ শুরু হয়েছে, তাই নিম্ন আয়ের মানুষ অর্থ ও সঞ্চয় সংকটে পড়বে। শুরুতেই সঞ্চয়হীন ভাসমান মানুষ, দিনমজুর, 
বৃদ্ধ-অনাথ-এতিম, রিকশা, ছোট কারখানা, নির্মাণশ্রমিক যাঁরা ‘দিন আনে দিন খান’, তাঁরা লকডাউনের দ্বিতীয় সপ্তাহ থেকেই আয় হীনতার কারণে 
খাদ্যের সংকটে পড়বেন। শহরের ভাসমান প্রান্তিক অর্থনৈতিক শ্রেণি সামাজিক উৎস থেকে ধার-ঋণ নিতে অক্ষম বলে তাদের জন্য খাদ্যসংকট অবধারিত। 
গ্রামে ‘সমাজের’ উপস্থিতি এবং কৃষি ও ক্ষুদ্রশিল্পভিত্তিক ‘উৎপাদনব্যবস্থা’ রয়েছে বিধায় সেখানে খাদ্যসংকট কিছুটা দেরিতে আসবে। গ্রামে ভাসমানদের 
কর্মহীনতার তৃতীয় কিংবা চতুর্থ সপ্তাহ থেকে খাদ্যসংকট শুরু হতে পারে, তার আগে পর্যন্ত তাঁরা চেয়েচিন্তে চলতে পারবেন হয়তো। পরেই আসবে 
কর্মহীন নিম্নবিত্ত, যাদের কিছুটা সঞ্চয় ছিল—এমন শ্রেণি। তার পরে আসবে বেতন বন্ধ হয়ে সঞ্চয় ফুরিয়ে যাওয়া নিম্ন–মধ্যবিত্ত কিংবা মধ্যবিত্তও। 
এই সব কটি প্রান্তিক ধারার জন্য জরুরি খাদ্য সরবরাহ করার একটা দায় আছে সরকারের। ইতিমধ্যেই পশ্চিমবঙ্গ সরকার সাত কোটি মানুষের 
ছয় মাসের জরুরি খাদ্য সরবরাহের ঘোষণা দিয়েছে। অন্যদিকে কেরালার সরকার কুড়ি হাজার কোটি রুপির করোনা প্যাকেজ ঘোষণা করেছে, 
এসেছে বিদ্যুৎ বিলে ছাড়ের ঘোষণা। বাংলাদেশেও মাথাপিছু ন্যূনতম ‘ক্যালরি ধারণ’ ভিত্তিতে ভাসমান প্রান্তিক শ্রেণি, স্থায়ী বেকার, তাৎক্ষণিকভাবে 
কাজহীন, বেতন বন্ধ হয়ে পড়া, সঞ্চয় ফুরিয়ে যাওয়া শ্রেণির জন্য খাদ্য সরবরাহের বাধ্যবাধকতা তৈরি হয়েছে। আর্থিক সংখ্যায় রূপান্তর করলে দেখা 
যায়, সরকারের জন্য তৈরি হয়েছে বড় এক আর্থিক বোঝা!

""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence) # store the distinct words
word_to_ix = {word: i for i, word in enumerate(vocab)}
print(torch.LongTensor([1,2,3]))

[(['প্রথমেই', 'শুরু'], 'হয়েছে'), (['শুরু', 'হয়েছে'], 'স্বাস্থ্য'), (['হয়েছে', 'স্বাস্থ্য'], 'ও')]
tensor([1, 2, 3])


#### 2. Create and Train Model

In [5]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
print("Before Training : {}".format(model.embeddings(torch.LongTensor([1])))) # first few random word embeddings
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(200):
    total_loss = 0
    for context, target in trigrams:

        # Prepare the inputs to be passed to the model
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long) #turn the words into integer indices and wrap them in tensors

        # Accumulated Gradient must be zeroed out before computing gradient again
        model.zero_grad()

        # Forward pass(probably using __Callable__), returns log probabilities over next words
        log_probs = model(context_idxs)

        # Compute loss function
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))  #target word wrapped in a tensor

        # Let the gradient flow backward
        loss.backward()
        
        # Update the gradient
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    
print(losses)  # The loss decreased every iteration over the training data!

print("After Training : {}".format(model.embeddings(torch.LongTensor([1])))) # first few random word embeddings again

Before Training : tensor([[-1.5454, -0.2652,  0.0883, -0.6848,  1.9326,  0.6381, -0.0039, -0.1837,
         -0.3661,  0.0090]], grad_fn=<EmbeddingBackward>)
[1226.2994923591614, 1220.7042546272278, 1215.1553497314453, 1209.647138118744, 1204.1756258010864, 1198.7373700141907, 1193.3269543647766, 1187.937581062317, 1182.5674076080322, 1177.2122716903687, 1171.8662571907043, 1166.5256400108337, 1161.1879529953003, 1155.8527135849, 1150.5159466266632, 1145.174000263214, 1139.8196818828583, 1134.4527990818024, 1129.0716054439545, 1123.6770544052124, 1118.2629771232605, 1112.8236479759216, 1107.360211610794, 1101.8709995746613, 1096.3513269424438, 1090.802624464035, 1085.2220740318298, 1079.6060998439789, 1073.9528334140778, 1068.261400461197, 1062.5285024642944, 1056.7520174980164, 1050.9319562911987, 1045.0672054290771, 1039.1580142974854, 1033.2015960216522, 1027.1994044780731, 1021.1505630016327, 1015.0489404201508, 1008.8993911743164, 1002.698742389679, 996.4467918872833, 990.149286985